In [74]:
import time
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import pairwise_distances

Data Loading

In [75]:
ratings_train = pd.read_csv("./Data/rate_train.csv", low_memory=False)
ratings_test = pd.read_csv("./Data/rate_test.csv",low_memory=False)

In [76]:
datasets_train = ratings_train.dataset_id.unique()
model_train = ratings_train.model_id.unique()
datasets_test = ratings_test.dataset_id.unique()
model_test = ratings_test.model_id.unique()
meta_models = pd.read_csv("./Data/models_num.csv",low_memory=False)
models = meta_models.model_id.unique()

Rating Matrix

In [77]:
data_model_train_matrix = pd.DataFrame(index=datasets_train,columns=models)
data_model_test_matrix = pd.DataFrame(index=datasets_test,columns=model_test)

In [78]:
for row in ratings_train.itertuples():
    data_model_train_matrix.loc[row[1]][row[2]] = row[3]


In [79]:
for row in ratings_test.itertuples():
    data_model_test_matrix.loc[row[1]][row[2]] = row[3]
data_model_test_matrix = data_model_test_matrix.fillna(0)

Dataset Similarity Matrix

In [80]:
meta_datasets = pd.read_csv("./Data/datasets_v.csv",low_memory=False)
datasets = meta_datasets.data_id.unique()
meta_datasets = meta_datasets.loc[:,("v1","v2","v3","v4","v5","v6","v7","v8")]
scaler = StandardScaler()

# 对dataframe的数据进行标准化
scaled_data = scaler.fit_transform(meta_datasets)
# 将标准化后的数据转换为dataframe，并保留原始索引
scaled_df = pd.DataFrame(scaled_data, index=meta_datasets.index, columns=meta_datasets.columns)
meta_dataset_similarity = cosine_similarity(scaled_df.values.tolist())
meta_dataset_similarity = pd.DataFrame(meta_dataset_similarity,index=datasets,columns=datasets)

KNN sim

In [81]:
def cosine_similarity_func(ratings, user1, user2):
    # 找到两个用户共同评分的物品，并将这些评分放入一个向量中
    u1_ratings = ratings.loc[user1].dropna()
    u2_ratings = ratings.loc[user2].dropna()

    common_items = np.intersect1d(u1_ratings.index, u2_ratings.index).tolist()
    u1_common_ratings = u1_ratings.loc[common_items]
    u2_common_ratings = u2_ratings.loc[common_items]

    # 计算两个向量之间的余弦相似度
    if len(common_items) == 0:
        return 0
    else:
        cos_sim = np.dot(u1_common_ratings, u2_common_ratings) / (np.linalg.norm(u1_common_ratings) * np.linalg.norm(u2_common_ratings))
        return cos_sim

In [82]:
dataset_similarity = pd.DataFrame(index=datasets_train,columns=datasets_train)

In [83]:
def create_bipartite_adjacency_matrix(rating_matrix):
    n_users, n_items = rating_matrix.shape
    adjacency_matrix = np.zeros((n_users + n_items, n_users + n_items))
    adjacency_matrix[:n_users, n_users:] = rating_matrix
    adjacency_matrix[n_users:, :n_users] = rating_matrix.T
    return adjacency_matrix

In [84]:
def propagation_matrix(adjacency, lambda_):
    n = adjacency.shape[0]
    I = np.eye(n)
    # 将 NaN 视为 0
    adjacency = np.nan_to_num(adjacency)
    try:
        P = np.linalg.inv(I - lambda_ * adjacency)
    except np.linalg.LinAlgError:
        print("矩阵不可逆，无法计算传播矩阵")
        return None
    return P

In [85]:
def propagation_matrix_withWalkLength(adjacency_matrix, max_walk_length):
    adjacency_matrix = np.nan_to_num(adjacency_matrix)
    propagation_matrix = np.eye(adjacency_matrix.shape[0])
    sum_matrix = np.eye(adjacency_matrix.shape[0])

    for _ in range(max_walk_length):
        propagation_matrix = propagation_matrix @ adjacency_matrix
        sum_matrix += propagation_matrix

    return sum_matrix

In [86]:
start_time_train = time.time()

In [87]:
# 计算二分图邻接矩阵
bipartite_adjacency_matrix = create_bipartite_adjacency_matrix(data_model_train_matrix)

In [88]:
propagation_maxLength = propagation_matrix_withWalkLength(bipartite_adjacency_matrix, 4)
lambda_p = 0.8 # 超参数 λ，用于调整传播矩阵的强度
propagation_lambda = propagation_matrix(bipartite_adjacency_matrix, lambda_p)

In [89]:
# 提取用户-商品传播矩阵和商品-用户传播矩阵
n_users = data_model_train_matrix.shape[0]
user_item_propagation = propagation_maxLength[:n_users, n_users:]
item_user_propagation = propagation_maxLength[n_users:, :n_users]

In [90]:
# 计算 Random Walk Kernel
random_walk_kernel = np.dot(user_item_propagation, item_user_propagation)

In [91]:
def normalize_kernel(kernel_matrix):
    # 计算矩阵的最小值和最大值
    min_val = np.min(kernel_matrix)
    max_val = np.max(kernel_matrix)

    # 防止除数为零的情况
    if max_val == min_val:
        return np.zeros_like(kernel_matrix)

    # 将矩阵的值缩放到0和1之间
    normalized_kernel_matrix = (kernel_matrix - min_val) / (max_val - min_val)

    return normalized_kernel_matrix

In [92]:
normalized_kernel = normalize_kernel(random_walk_kernel)

In [93]:
normalized_kernel = pd.DataFrame(normalized_kernel)

In [94]:
normalized_kernel

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,0.003250,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.002027,0.000000,0.00000,0.000000
1,0.000000,1.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.992847,0.000000,0.839609,...,0.992723,0.0,0.0,0.0,0.0,0.000000,0.000000,0.862140,0.00000,0.000000
2,0.000000,0.00000,0.000268,0.000000,0.000000,0.0,0.000000,0.000000,0.000106,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000
3,0.000000,0.00000,0.000000,0.000571,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000573
4,0.000000,0.00000,0.000000,0.000000,0.046471,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.063893,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.064772,0.000000,0.000000,0.00000,0.000000
68,0.002027,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.001265,0.000000,0.00000,0.000000
69,0.000000,0.86214,0.000000,0.000000,0.000000,0.0,0.000000,0.855974,0.000000,0.723917,...,0.855867,0.0,0.0,0.0,0.0,0.000000,0.000000,0.743345,0.00000,0.000000
70,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00012,0.000000


In [95]:
lambda_ = 0.5
for i in datasets_train:
    for j in datasets_train:
        if normalized_kernel.loc[i][j] != 0:
            dataset_similarity.loc[i][j] = normalized_kernel.loc[i][j]
        else:
            dataset_similarity.loc[i][j] = 0


In [96]:
dataset_similarity

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,0.00325,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.002027,0,0,0
1,0,1.0,0,0,0,0,0,0.992847,0,0.839609,...,0.992723,0,0,0,0,0,0,0.86214,0,0
2,0,0,0.000268,0,0,0,0,0,0.000106,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0.000571,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000573
4,0,0,0,0,0.046471,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0,0,0,0,0,0,0.063893,0,0,0,...,0,0,0,0,0,0.064772,0,0,0,0
68,0.002027,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.001265,0,0,0
69,0,0.86214,0,0,0,0,0,0.855974,0,0.723917,...,0.855867,0,0,0,0,0,0,0.743345,0,0
70,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.00012,0


Prediction

In [97]:
def predict_ratings(rating_matrix, user_similarity_matrix):
    """
    输入：
    rating_matrix - 评分矩阵，DataFrame格式，其中NaN表示未评分
    user_similarity_matrix - 用户相似度矩阵，DataFrame格式
    k - 最近邻的数量，默认为5

    输出：
    prediction_matrix - 预测矩阵，DataFrame格式
    """

    # 初始化预测矩阵
    prediction_matrix = rating_matrix.copy()

    # 对于评分矩阵中的每个NaN值，使用K最近邻的方法预测评分
    for i in range(rating_matrix.shape[0]):
        for j in range(rating_matrix.shape[1]):
            if np.isnan(rating_matrix.iloc[i, j]):
                # 获取第i个用户的相似度值，并在相似度矩阵中找到K个最相似的用户
                similarity_values = user_similarity_matrix.iloc[i].sort_values(ascending=False)[1:]

                # 计算加权平均评分
                weighted_sum = 0
                similarity_sum = 0
                for index, value in similarity_values.items():
                    user_rating = rating_matrix.iloc[index, j]
                    if not np.isnan(user_rating):
                        weighted_sum += value * user_rating
                        similarity_sum += value

                # 如果存在至少一个相似用户对该物品进行了评分，则计算预测评分
                if similarity_sum != 0:
                    prediction_matrix.iloc[i, j] = weighted_sum / similarity_sum
                else:
                    # 如果没有相似用户评分，则使用当前用户的平均评分作为预测值
                    prediction_matrix.iloc[i, j] = rating_matrix.iloc[i].mean()

    return prediction_matrix

In [98]:
def predict(rating_matrix, similarity_matrix):
    """
    根据评分矩阵和相似度矩阵预测评分。

    参数：
    rating_matrix (pd.DataFrame)：评分矩阵，包含NaN值
    similarity_matrix (numpy.array)：相似度矩阵

    返回：
    pd.DataFrame：预测评分矩阵
    """

    # 获取评分矩阵的均值（忽略NaN值）
    mean_rating = rating_matrix.mean(axis=1).values

    # 将评分矩阵中的NaN值替换为0
    rating_matrix_nan_to_zero = rating_matrix.fillna(0).values

    # 减去均值，得到归一化的评分矩阵
    normalized_rating_matrix = rating_matrix_nan_to_zero - mean_rating[:, np.newaxis]

    # 计算预测评分
    predicted_ratings = mean_rating[:, np.newaxis] + np.dot(similarity_matrix, normalized_rating_matrix) / np.abs(similarity_matrix).sum(axis=1)[:, np.newaxis]

    # 将预测评分数组转换为DataFrame
    predicted_ratings_df = pd.DataFrame(predicted_ratings, index=rating_matrix.index, columns=rating_matrix.columns)

    return predicted_ratings_df

In [99]:
model_prediction_train = predict(data_model_train_matrix,dataset_similarity)
model_prediction_train = pd.DataFrame(model_prediction_train,index=datasets_train,columns=models).sort_index().sort_index(axis=1)

c:\users\byy\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


In [100]:
model_prediction_train

,72,73,74,75,76,77,78,79,80,81,...,1862,1863,1864,1865,1866,1867,1868,1869,1870,1871
0,0.6117,0.587688,0.556949,0.786475,0.757494,0.724982,0.726397,0.744265,0.790785,0.768843,...,0.038728,0.038728,0.038728,0.038728,0.038728,0.038728,0.038728,0.038728,0.038728,0.038728
1,0.020827,0.020827,0.020827,0.020827,0.020827,0.020827,0.020827,0.020827,0.020827,0.020827,...,0.020827,0.020827,0.020827,0.020827,0.020827,0.020827,0.020827,0.020827,0.020827,0.020827
2,-0.10415,-0.10415,-0.10415,-0.10415,-0.10415,-0.10415,-0.10415,-0.10415,-0.10415,-0.10415,...,-0.10415,-0.10415,-0.10415,-0.10415,-0.10415,-0.10415,-0.10415,-0.10415,-0.10415,-0.10415
3,-0.090424,-0.090424,-0.090424,-0.090424,-0.090424,-0.090424,-0.090424,-0.090424,-0.090424,-0.090424,...,0.040805,0.13622,0.071826,0.070316,0.07137,0.073026,0.07137,0.15315,0.135254,0.073357
4,-0.033299,-0.033299,-0.033299,-0.033299,-0.033299,-0.033299,-0.033299,-0.033299,-0.033299,-0.033299,...,-0.033299,-0.033299,-0.033299,-0.033299,-0.033299,-0.033299,-0.033299,-0.033299,-0.033299,-0.033299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.025903,0.025903,0.025903,0.025903,0.025903,0.025903,0.025903,0.025903,0.025903,0.025903,...,0.025903,0.025903,0.025903,0.025903,0.025903,0.025903,0.025903,0.025903,0.025903,0.025903
68,0.480995,0.456979,0.426236,0.65581,0.626817,0.594292,0.595705,0.613579,0.660117,0.638167,...,-0.09183,-0.09183,-0.09183,-0.09183,-0.09183,-0.09183,-0.09183,-0.09183,-0.09183,-0.09183
69,-0.028359,-0.028359,-0.028359,-0.028359,-0.028359,-0.028359,-0.028359,-0.028359,-0.028359,-0.028359,...,-0.028359,-0.028359,-0.028359,-0.028359,-0.028359,-0.028359,-0.028359,-0.028359,-0.028359,-0.028359
70,-0.010546,-0.010546,-0.010546,-0.010546,-0.010546,-0.010546,-0.010546,-0.010546,-0.010546,-0.010546,...,-0.010546,-0.010546,-0.010546,-0.010546,-0.010546,-0.010546,-0.010546,-0.010546,-0.010546,-0.010546


In [101]:
model_prediction_test = pd.DataFrame(index=datasets_test,columns=model_test)

In [102]:
end_time_train = time.time()
Training_time = end_time_train - start_time_train
Training_time

1.7665512561798096

In [103]:
def find_sim_index(index):
    row1 = meta_dataset_similarity.loc[index]
    row1_max_index = row1[row1 == row1.max()].index[0]
    return row1_max_index

In [134]:
def Find_Top_k(i,sim_matrix):
    row = sim_matrix.loc[i]
    row = row.sort_values(ascending=False)
    index_row = row.index
    index_row = index_row.values.tolist()
    return index_row

In [135]:
start_time = time.time()

In [136]:
for dataset in datasets_test:
    for model in model_test:
        dataset_sim_list = Find_Top_k(dataset,meta_dataset_similarity)[1:]
        # 仅保留存在于 model_prediction_train 的索引
        valid_indices = [idx for idx in dataset_sim_list if idx in model_prediction_train.index][:15]
        model_prediction_test.loc[dataset][model] = model_prediction_train.loc[valid_indices][model].mean()

In [137]:
end_time = time.time()

In [138]:
end_time - start_time

5.880357503890991

In [139]:
for i in datasets_test:
    for j in model_test:
        if data_model_test_matrix.loc[i][j] == 0:
            model_prediction_test.loc[i][j] = None

In [140]:
result = pd.DataFrame(columns={"dataset","model","balanced_accuracy","groundtruth_balanced_accuracy"})

In [141]:
for i in datasets_test:
    for j in model_test:
        if model_prediction_test.loc[i][j] is not None:
            balanced_accuracy = model_prediction_test.loc[i][j]
            groundtruth_balanced_accuracy = data_model_test_matrix.loc[i][j]
            result = result.append([{'dataset':i,'model':j,'balanced_accuracy':balanced_accuracy,'groundtruth_balanced_accuracy':groundtruth_balanced_accuracy}],ignore_index=True)

In [142]:
result

,model,balanced_accuracy,groundtruth_balanced_accuracy,dataset
0,97,0.168344,0.672734,1
1,98,0.174071,0.748037,1
2,99,0.179798,0.755532,1
3,100,0.219836,0.995360,1
4,101,0.214112,0.994290,1
...,...,...,...,...
1370,1767,0.112667,0.500000,67
1371,1768,0.112716,0.500000,67
1372,1769,0.120068,0.577401,67
1373,1770,0.113074,0.500000,67


In [143]:
result.to_csv("../Kaggle/Output/RandomWalk_only/Full_RandomWalk_only@4@15.csv",index=False)